In [46]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from skopt.space import Integer
from numpy import mean
from skopt.utils import use_named_args
import numpy as np
#
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, median_absolute_error


In [ ]:
#dataset = pd.read_csv('D:/Code/Python/NCKH/A1/c.csv')
dataset = pd.read_csv('./Dataset/para.csv')
# 
features = ["Freq"]
targets = ["W", "L", "Lf"]
x = dataset[features]
y = dataset[targets]

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [49]:
search_space = [Integer(1, 100, name='max_depth'), Integer(2, 100, name='min_samples_split')]

In [ ]:
@use_named_args(search_space)
def evaluate_model(**params):
    model = RandomForestRegressor(max_depth=params['max_depth'], min_samples_split=params['min_samples_split'])
    model.fit(x_train_scaled, y_train)
    y_pred = model.predict(x_test_scaled)
    error = mean((y_test - y_pred) ** 2)  # MSE
    return error

In [51]:
result = gp_minimize(evaluate_model, search_space)

C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [20, 2] before, using random point [76, 86]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [20, 2] before, using random point [28, 97]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [20, 2] before, using random point [8, 72]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages

In [52]:
best_max_depth = result.x[0]
best_min_samples_split = result.x[1]
best_model = RandomForestRegressor(max_depth=best_max_depth, min_samples_split=best_min_samples_split)
best_model.fit(x_train_scaled, y_train)

RandomForestRegressor(max_depth=18)

In [ ]:
new_dataset = pd.read_csv('./Dataset/para.csv')  # 
new_data = new_dataset["Freq"].values.reshape(-1, 1)  # 

# Scaling 
new_data_scaled = scaler.transform(new_data)

# 
y_pred = best_model.predict(x_test_scaled)

C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: max_depth=%d, min_samples_split=%d' % (result.x[0], result.x[1]))

# 

print("Predictions:", y_pred)

Best Accuracy: 0.998
Best Parameters: max_depth=18, min_samples_split=2
Predictions: [[ 25.5674  19.5783  11.5176]
 [ 49.2456  38.2779  22.1843]
 [109.888   85.9293  49.5187]
 [ 38.392   29.7251  17.2955]
 [ 25.468   19.4994  11.4704]
 [ 17.1719  12.8942   7.7348]
 [ 26.8567  20.6057  12.0976]
 [ 44.8371  34.8095  20.2013]
 [ 29.3945  22.6097  13.2424]
 [ 43.2687  33.57    19.4935]
 [ 38.7719  30.0183  17.4633]
 [100.5648  78.5957  45.3062]
 [ 35.5155  27.4512  15.9983]
 [ 39.2109  30.3674  17.6608]
 [ 24.7145  18.904   11.1335]
 [ 95.2112  74.3939  42.8956]
 [ 27.1117  20.8018  12.2141]
 [ 37.7935  29.2495  17.0255]
 [ 28.018   21.5206  12.6227]
 [ 25.5143  19.5347  11.4949]
 [ 49.8022  38.7137  22.4374]
 [ 35.353   27.3234  15.9258]
 [ 17.3504  13.0401   7.8138]
 [ 17.1321  12.8626   7.7184]
 [ 51.4582  40.0226  23.1811]
 [ 16.606   12.4436   7.4815]
 [ 34.0085  26.2678  15.3189]
 [107.483   84.0049  48.4128]
 [103.296   80.738   46.5328]
 [ 18.0212  13.5755   8.1165]
 [ 34.3882  26.

In [ ]:
FreqInput = float(input('Nhập vào tần số muốn dự đoán: '))

# 
new_data = [[FreqInput]]

# Scaling 
new_data_scaled = scaler.transform(new_data)

# 
y_pred = best_model.predict(new_data_scaled)

# 
print('Giá trị nhập vào là: ', FreqInput)
print('Giá trị dự đoán:', y_pred)

In [ ]:
def regression_report(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    explained_variance = explained_variance_score(y_true, y_pred)
    median_ae = median_absolute_error(y_true, y_pred)

    # Mean Bias Deviation (MBD)
    mbd = np.mean(y_pred - y_true)

    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Symmetric Mean Absolute Percentage Error (sMAPE)
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

    report = pd.DataFrame({
        'Metric': [
            'Mean Absolute Error (MAE)',
            'Mean Squared Error (MSE)',
            'Root Mean Squared Error (RMSE)',
            'R² Score',
            'Explained Variance Score',
            'Median Absolute Error',
            'Mean Bias Deviation (MBD)',
            'Mean Absolute Percentage Error (MAPE)',
            'Symmetric Mean Absolute Percentage Error (sMAPE)'
        ],
        'Gausians': [mae, mse, rmse, r2, explained_variance, median_ae, mbd, mape, smape]
    })

    return report

report = regression_report(y_test, y_pred)
print(report)

                                             Metric  Gausians
0                         Mean Absolute Error (MAE)  0.021919
1                          Mean Squared Error (MSE)  0.002097
2                    Root Mean Squared Error (RMSE)  0.045789
3                                          R² Score  0.999995
4                          Explained Variance Score  0.999995
5                             Median Absolute Error  0.009383
6                         Mean Bias Deviation (MBD)  0.007028
7             Mean Absolute Percentage Error (MAPE)  0.055001
8  Symmetric Mean Absolute Percentage Error (sMAPE)  0.054990
